In [1]:
import numpy as np
import numba as nb

import ice_funcs as icef

## Todo:
- ***Make sure that diffusion works!!***
- ***Figure out if I actually need to hold on to timings... and if yes... how ?***
    - Is the timing array even needed &&&**[do not delete, just comment out]**&&&
- Make relaxSaturationUtilities fully OOP


## Backburner-type-beat:
- make sure everything is numbafied
- Continue commenting in `ice_funcs.py`

## Improvements
- Monopole matching
- correct geometry factors $G_b$ and $H_b$
- Causal timings?

In [2]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [3]:
gu = icef.GeneralUtilities(101) # general utilities
pu = icef.PhysicsUtilities(0.01, 10, X_0=1) # physics utilities
sru = icef.SaturationRelaxationUtilities(101, 200)

In [4]:
gru = icef.GrowthUtilities(101)

In [5]:
SnowflakeSim = icef.SnowflakeSimulation(101, 200)
# SnowflakeSim.GeneralU.neighbor_array

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1mFailed in nopython mode pipeline (step: nopython frontend)
[1m[1m[1m[1m- Resolution failure for literal arguments:
[1mFailed in nopython mode pipeline (step: native lowering)
LLVM IR parsing error
<string>:91:201: error: base element of getelementptr must be sized
  %".13" = getelementptr inbounds {i32, i32, double, i32, i32, {i8*, i8*, i64, i64, i8*, [2 x i64], [2 x i64]}, {i8*, i8*, i64, i64, double*, [2 x i64], [2 x i64]}, %"deferred.139748488724096.data"}, {i32, i32, double, i32, i32, {i8*, i8*, i64, i64, i8*, [2 x i64], [2 x i64]}, {i8*, i8*, i64, i64, double*, [2 x i64], [2 x i64]}, %"deferred.139748488724096.data"}* %".12", i32 0, i32 0
                                                                                                                                                                                                        ^
[0m
[0m[1m- Resolution failure for non-literal arg

In [ ]:
neighbor_array_test = gu.neighbor_array
diffusion_rules_test = sru.diffusion_rules

print(diffusion_rules_test)

[[[-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  [-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  [-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  ...
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 0.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00]]

 [[ 1.6666667e-01  3.3333334e-01  3.3333334e-01  1.6666667e-01
    0.0000000e+00  0.0000000e+00]
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  ...
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  [ 1.6666667e-0

In [ ]:
default_ice_map = np.full((101, 101), False)
default_ice_map[-1,0] = True
default_ice_map[-2,0] = True
default_ice_map[-3,0] = True
default_ice_map[-3,1] = True

# %timeit test_boundary_map
boundary_map_test = gu.construct_boundary_map(default_ice_map)
print(boundary_map_test[-5:,:5])

sat_map_test = np.full((101,101), 1, dtype=np.float32)

normal_cells_test, boundary_cells_test = gu._distinguish_cells(default_ice_map, boundary_map_test)

opp_array_test = sru._construct_opp_array(boundary_cells_test, default_ice_map, neighbor_array_test)

[[0 0 0 0 0]
 [1 2 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [ ]:
# pu.apply_boundary_condition(2, 2, sat_map_test, 0.5, boundary_map_test)

sat_map_test = np.full((101,101), 1, dtype=np.float32)

# for i in range(50):
#     sru._execute_relaxation_step(sat_map_test, normal_cells_test, boundary_cells_test, opp_array_test, neighbor_array_test, boundary_map_test)

new_thang = sru.diffuse_to_convergence(sat_map_test, 200, 0.0001, normal_cells_test, boundary_cells_test, default_ice_map, boundary_map_test, neighbor_array_test, pu)
print(new_thang[1])
print(new_thang[2])
print(new_thang[0][-10:,:5])


%timeit sru.diffuse_to_convergence(sat_map_test, 200, 0.001, normal_cells_test, boundary_cells_test, default_ice_map, boundary_map_test, neighbor_array_test, pu)

True
50
[[0.99865556 0.99835116 0.99807584 0.99787086 0.99777734]
 [0.99790716 0.99746865 0.99709535 0.9968527  0.99677205]
 [0.9968196  0.99620247 0.9957176  0.995485   1.        ]
 [0.9952684  0.9944138  0.99380445 0.99358547 1.        ]
 [0.9930891  0.991905   0.9912093  1.         1.        ]
 [0.99010193 0.98826534 0.98723716 1.         1.        ]
 [0.98661655 0.9819473  1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]]
292 µs ± 6.55 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
gru.update_minimum_time(boundary_cells_test, sat_map_test, boundary_map_test, pu)
gru.minimum_time_increment

gru.update_filling_array(boundary_cells_test, sat_map_test, boundary_map_test, pu)
gru.filling_array[-5:,:5]

array([[0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.60653067, 1.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)